In [1]:
%matplotlib inline
from IPython.display import Image, HTML
import json
import datetime
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS
import plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
warnings.filterwarnings('ignore')
plotly.tools.set_credentials_file(username='rounakbanik', api_key='xTLaHBy9MVv5szF4Pwan')
plt.rcParams['font.sans-serif']=['SimHei'] # 用来正常显示中文标签 
plt.rcParams['axes.unicode_minus']=False # 用来正常显示正负号
#sns.set_style('whitegrid')
#sns.set(font_scale=1.25)
pd.set_option('display.max_colwidth', 50)

In [2]:
# 导入数据
df = pd.read_excel('/Users/chandler/Desktop/618_bhsm/jihe/2018.xlsx')
df.head()

推送类型   渠道       发送日期       数据日期 活动类型              活动编号              活动名称  \
0   精推  站内信 2018-02-05 2018-02-07    -  C0004_2018/02/05  C0004_2018/02/05   
1   精推  站内信 2018-03-05 2018-03-07    -  C0004_2018/03/05  C0004_2018/03/05   
2   精推  站内信 2018-02-05 2018-02-07    -  C0005_2018/02/05  C0005_2018/02/05   
3   精推  站内信 2018-03-05 2018-03-07    -  C0005_2018/03/05  C0005_2018/03/05   
4   精推  站内信 2018-01-01 2018-01-03    -  C0006_2018/01/01  C0006_2018/01/01   

  活动归属编码 活动归属名称    计划发送会员数     ...           PV  加入购物车UV 加入购物车转化率  提交订单数  \
0      -      -  1953476.0     ...       2136.0      0.0      0.0  153.0   
1      -      -  2227871.0     ...       1790.0      0.0      0.0   69.0   
2      -      -   369542.0     ...        421.0      0.0      0.0   26.0   
3      -      -   747191.0     ...        721.0      0.0      0.0   32.0   
4      -      -     1875.0     ...         65.0      0.0      0.0    1.0   

      提交订单金额  下单会员数   买家数  成功支付订单数      付款金额          客单价  
0  137519.09   76.0  72.0    106.0  74440.94  1033.901944  
1   85963.94   46.0  44.0     55.0  50043.27  1137.347045  
2   20534.68   10.0  10.0     26.0  19909.71  1990.971000  
3  114552.62   16.0  15.0     24.0  64011.76  4267.450667  
4      11.90    1.0   1.0      1.0      1.21     1.210000  

[5 rows x 53 columns]

In [3]:
df.columns

Index(['推送类型', '渠道', '发送日期', '数据日期', '活动类型', '活动编号', '活动名称', '活动归属编码',
       '活动归属名称', '计划发送会员数', '发送成功会员数', '发送成功率', 'A-打开率', 'B-点击率', '人均下单数',
       '下单会员数1', '买家数1', '成功支付订单数1', '付款金额1', '客单价1', 'C-转化率1', '下单会员数2',
       '买家数2', '成功支付订单数2', '付款金额2', '客单价2', 'D-目标转化率2', '下单会员数3', '买家数3',
       '成功支付订单数3', '付款金额3', '客单价3', 'C-转化率3', '下单会员数4', '买家数4', '成功支付订单数4',
       '付款金额4', '客单价4', 'D-目标转化率4', '反馈会员数', '打开会员数', '点击会员数', 'UV', 'PV',
       '加入购物车UV', '加入购物车转化率', '提交订单数', '提交订单金额', '下单会员数', '买家数', '成功支付订单数',
       '付款金额', '客单价'],
      dtype='object')

In [4]:
df = df[['发送日期', '推送类型', '渠道', '活动编号', '计划发送会员数', '发送成功会员数', '买家数1', '买家数3', '打开会员数', 'UV']]

In [5]:
df.head(20)

发送日期 推送类型   渠道              活动编号    计划发送会员数  发送成功会员数     买家数1   买家数3  \
0  2018-02-05   精推  站内信  C0004_2018/02/05  1953476.0      NaN  13352.0  235.0   
1  2018-03-05   精推  站内信  C0004_2018/03/05  2227871.0      NaN  13922.0  197.0   
2  2018-02-05   精推  站内信  C0005_2018/02/05   369542.0      NaN   2381.0   40.0   
3  2018-03-05   精推  站内信  C0005_2018/03/05   747191.0      NaN   6336.0   47.0   
4  2018-01-01   精推  站内信  C0006_2018/01/01     1875.0      NaN    229.0    2.0   
5  2018-01-02   精推  站内信  C0006_2018/01/02     2132.0      NaN    298.0    2.0   
6  2018-01-03   精推  站内信  C0006_2018/01/03     1844.0      NaN    206.0    4.0   
7  2018-01-04   精推  站内信  C0006_2018/01/04     2257.0      NaN    300.0    1.0   
8  2018-01-05   精推  站内信  C0006_2018/01/05     1708.0      NaN    228.0    6.0   
9  2018-01-06   精推  站内信  C0006_2018/01/06     1930.0      NaN    275.0    5.0   
10 2018-01-07   精推  站内信  C0006_2018/01/07     1782.0      NaN    304.0    3.0   
11 2018-01-08   精推  站内信  C0006_2018/01/08     1677.0      NaN    257.0    4.0   
12 2018-01-09   精推  站内信  C0006_2018/01/09     2279.0      NaN    311.0    4.0   
13 2018-01-11   精推  站内信  C0006_2018/01/11     1877.0      NaN    223.0    1.0   
14 2018-01-12   精推  站内信  C0006_2018/01/12     2000.0      NaN    258.0    3.0   
15 2018-01-13   精推  站内信  C0006_2018/01/13     1736.0      NaN    210.0    2.0   
16 2018-01-14   精推  站内信  C0006_2018/01/14     2158.0      NaN    340.0    4.0   
17 2018-01-15   精推  站内信  C0006_2018/01/15     1743.0      NaN    297.0    3.0   
18 2018-01-16   精推  站内信  C0006_2018/01/16      139.0      NaN     19.0    1.0   
19 2018-01-17   精推  站内信  C0006_2018/01/17     1715.0      NaN    304.0    6.0   

    打开会员数     UV  
0   548.0  173.0  
1   523.0  139.0  
2    96.0   33.0  
3   113.0   46.0  
4     7.0    5.0  
5     4.0    3.0  
6    10.0    6.0  
7    10.0    6.0  
8    10.0    3.0  
9     5.0    4.0  
10    5.0    3.0  
11    6.0    2.0  
12   11.0    4.0  
13    8.0    4.0  
14    6.0    5.0  
15    5.0    4.0  
16    7.0    2.0  
17    5.0    3.0  
18    1.0    1.0  
19    9.0    1.0

In [15]:
# 循环遍历df的所有索引
for i in df.index:
    if '_' in df['活动编号'].iat[i]:
        list = df['活动编号'].iat[i].split('_')
        df['活动编号'].iat[i] = list[0]

In [26]:
df['类别'] = 'LB'
df['场景'] = 'CJ'
df = df.reset_index(drop=True)
df.head(20)

发送日期 推送类型   渠道   活动编号    计划发送会员数  发送成功会员数     买家数1   买家数3  打开会员数  \
0  2018-02-05   精推  站内信  C0004  1953476.0      NaN  13352.0  235.0  548.0   
1  2018-03-05   精推  站内信  C0004  2227871.0      NaN  13922.0  197.0  523.0   
2  2018-02-05   精推  站内信  C0005   369542.0      NaN   2381.0   40.0   96.0   
3  2018-03-05   精推  站内信  C0005   747191.0      NaN   6336.0   47.0  113.0   
4  2018-01-01   精推  站内信  C0006     1875.0      NaN    229.0    2.0    7.0   
5  2018-01-02   精推  站内信  C0006     2132.0      NaN    298.0    2.0    4.0   
6  2018-01-03   精推  站内信  C0006     1844.0      NaN    206.0    4.0   10.0   
7  2018-01-04   精推  站内信  C0006     2257.0      NaN    300.0    1.0   10.0   
8  2018-01-05   精推  站内信  C0006     1708.0      NaN    228.0    6.0   10.0   
9  2018-01-06   精推  站内信  C0006     1930.0      NaN    275.0    5.0    5.0   
10 2018-01-07   精推  站内信  C0006     1782.0      NaN    304.0    3.0    5.0   
11 2018-01-08   精推  站内信  C0006     1677.0      NaN    257.0    4.0    6.0   
12 2018-01-09   精推  站内信  C0006     2279.0      NaN    311.0    4.0   11.0   
13 2018-01-11   精推  站内信  C0006     1877.0      NaN    223.0    1.0    8.0   
14 2018-01-12   精推  站内信  C0006     2000.0      NaN    258.0    3.0    6.0   
15 2018-01-13   精推  站内信  C0006     1736.0      NaN    210.0    2.0    5.0   
16 2018-01-14   精推  站内信  C0006     2158.0      NaN    340.0    4.0    7.0   
17 2018-01-15   精推  站内信  C0006     1743.0      NaN    297.0    3.0    5.0   
18 2018-01-16   精推  站内信  C0006      139.0      NaN     19.0    1.0    1.0   
19 2018-01-17   精推  站内信  C0006     1715.0      NaN    304.0    6.0    9.0   

       UV  类别  场景  
0   173.0  LB  CJ  
1   139.0  LB  CJ  
2    33.0  LB  CJ  
3    46.0  LB  CJ  
4     5.0  LB  CJ  
5     3.0  LB  CJ  
6     6.0  LB  CJ  
7     6.0  LB  CJ  
8     3.0  LB  CJ  
9     4.0  LB  CJ  
10    3.0  LB  CJ  
11    2.0  LB  CJ  
12    4.0  LB  CJ  
13    4.0  LB  CJ  
14    5.0  LB  CJ  
15    4.0  LB  CJ  
16    2.0  LB  CJ  
17    3.0  LB  CJ  
18    1.0  LB  CJ  
19    1.0  LB  CJ

In [27]:
# 导入数据
df_fht = pd.read_excel('/Users/chandler/Documents/Projects/sndataclean/fenghuotai/fenghuotai_cj.xlsx')
df_fht = df_fht.reset_index(drop=True)
df_fht.head()

场景编码        活动发起方     类别  消息类别         场景
0  A0001  运营总部-会员管理中心  核心购物类  精选活动    搜索浏览未购物
1  A0002  运营总部-会员管理中心  核心购物类  精选活动   加入购物车未购物
2  A0003  运营总部-会员管理中心  核心购物类  精选活动       买了还买
3  A0004  运营总部-会员管理中心  核心购物类  精选活动    浏览促销页推荐
4  A0005  运营总部-会员管理中心  核心购物类  通知服务  购物车商品降价提醒

In [29]:
# 循环遍历df的所有索引
for i in df.index:
    # 判断如果是短信渠道就要删除'买家数4', '打开会员数', 'UV'这三个列的所有数据并赋值为0
    act_code = df['活动编号'].iloc[i]
    for j in df_fht.index:
        fht_code = df_fht['场景编码'].iloc[j]
        if act_code == fht_code:
            df['类别'].iat[i] = df_fht['类别'].iat[j]
            df['场景'].iat[i] = df_fht['场景'].iat[j]

In [31]:
df.head(10)

发送日期 推送类型   渠道   活动编号    计划发送会员数  发送成功会员数     买家数1   买家数3  打开会员数  \
0 2018-02-05   精推  站内信  C0004  1953476.0      NaN  13352.0  235.0  548.0   
1 2018-03-05   精推  站内信  C0004  2227871.0      NaN  13922.0  197.0  523.0   
2 2018-02-05   精推  站内信  C0005   369542.0      NaN   2381.0   40.0   96.0   
3 2018-03-05   精推  站内信  C0005   747191.0      NaN   6336.0   47.0  113.0   
4 2018-01-01   精推  站内信  C0006     1875.0      NaN    229.0    2.0    7.0   
5 2018-01-02   精推  站内信  C0006     2132.0      NaN    298.0    2.0    4.0   
6 2018-01-03   精推  站内信  C0006     1844.0      NaN    206.0    4.0   10.0   
7 2018-01-04   精推  站内信  C0006     2257.0      NaN    300.0    1.0   10.0   
8 2018-01-05   精推  站内信  C0006     1708.0      NaN    228.0    6.0   10.0   
9 2018-01-06   精推  站内信  C0006     1930.0      NaN    275.0    5.0    5.0   

      UV     类别                     场景  
0  173.0  会员权益类  小额云钻过期提醒（月度事件，每月5号发送）  
1  139.0  会员权益类  小额云钻过期提醒（月度事件，每月5号发送）  
2   33.0  会员权益类  大额云钻过期提醒（月度事件，每月5号发送）  
3   46.0  会员权益类  大额云钻过期提醒（月度事件，每月5号发送）  
4    5.0  会员权益类           生日前六天领券祝福_V4  
5    3.0  会员权益类           生日前六天领券祝福_V4  
6    6.0  会员权益类           生日前六天领券祝福_V4  
7    6.0  会员权益类           生日前六天领券祝福_V4  
8    3.0  会员权益类           生日前六天领券祝福_V4  
9    4.0  会员权益类           生日前六天领券祝福_V4

In [ ]:
# 循环遍历df的所有索引
for i in df.index:
    # 判断如果是短信渠道就要删除'买家数4', '打开会员数', 'UV'这三个列的所有数据并赋值为0
    if df['渠道'].iloc[i] == '短信':
        df['打开会员数'].iat[i] = 0
        df['UV'].iat[i] = 0
        df['买家数4'].iat[i] = 0
    # 否则就要删除'计划发送会员数', '买家数2'这二个列的所有数据并赋值为0
    if df['渠道'].iloc[i] != '短信':
        df['计划发送会员数'].iat[i] = 0
        df['买家数2'].iat[i] = 0

# 整个df只要活动归属名称在act_belongto_name这个列表中的所有数据
df = df[df['活动归属名称'].isin(act_belongto_name)]
# 对处理后的dataframe重新索引（删除一些数据，索引可能出错，比如出现0，1，2，3，12，13这样的断点）

In [ ]:
pwd